# Student performance project

In [3]:
!cp /content/drive/MyDrive/U/Proyectos-personales/ML/1-Student-performance/student-mat.csv /content/

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import copy, math
import sklearn
from sklearn import linear_model
from sklearn.utils import shuffle

sns.set(style='darkgrid', palette='twilight_shifted')

During this project, I will create a simple regression model to predict students performance given general information about them.

This project will contain a numpy handmade solution and a sklearn solution.

In [2]:
df = pd.read_csv('./student-mat.csv')
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


We take only some features. In this occasion, we take the most relevant numeric fratures, chosen after visualizing them.

In [3]:
df = df[['traveltime', 'studytime', 'failures', 'absences', 'G1', 'G2', 'G3']]
df

,traveltime,studytime,failures,absences,G1,G2,G3
0,2,2,0,6,5,6,6
1,1,2,0,4,5,5,6
2,1,2,3,10,7,8,10
3,1,3,0,2,15,14,15
4,1,2,0,4,6,10,10
...,...,...,...,...,...,...,...
390,1,2,2,11,9,9,9
391,2,1,0,3,14,16,16
392,1,1,3,3,10,8,7
393,3,1,0,0,11,12,10


Generation of train and test datasets

In [32]:
X = np.array(df.drop(["G3"], axis=1))
y = np.array(df["G3"])

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.1)
print(y_train)

[12  0 12  0 12 14 10 18 10 11 15  5 11  0  8  8  0 12 13 15 17 16 11  0
 11 10 13 11 15  5  9 11 10 13 18 12 10 13  9 12  0 15  8 11  9  6 11  8
 11  9 18 10 10 17 11 10 16  0 13 11  7 11 18 16 15 19  7  8 19 14 11 10
 10 18  9 13  0 16 15 11  8 14 11 14 15 11 10  0  4  9  8 15 14 12  9 19
  8 15 13 13 16 18 10  7 15  5  8  9 18 11  7 13 13 17  6  8  7 10  6 14
 14 10 15  9 11 10 13 12 12 11 13  0 18  9 16  7 15 10  0  9  6 11 15 15
 15 12  8  9 14  0 13 14 10 14 15 13  9 10 12 13 12  8  6  5  8 15  9 15
 12 10  0 12 16 11 11  8 11  0 11 13 13 11 14 10 15  0 14 17  8  9 11  6
  8 10 10 11  9 15 11 16  9 12  0 16 20  0 10  6 11 12 12 18  0 11 16  6
 16 14  8 13 13 10  0  8  9 10 15  6 10 14 15  8  8  8 16  5 11  0 18  0
  8  6 10 12  6 11 12  7 12 10  8  7 14 14  9 11  8  0 12 11 10 19  5 14
 10 14 11 14  5  9  0  0 10  0 12  0 14 15 15 13 12 14  8 16  0  9 10  9
  0 15  0  8  0 10  9 15 10 16 13  8 13  9 15 15 11 11 14 15 11 10  0 10
 14  6 16 10  9  9 11 10  0  0 11 13  8  8 11 10 15

# Numpy implementation

In [74]:
def compute_cost(X, y, w, b):
    m, n = X.shape
    cost = 0.0

    for i in range(m):
        cost += ((np.dot(X[i], w) + b) - y[i])**2
    cost /= (2 * m)   
    
    return cost

def compute_gradient(X, y, w, b):
    m, n = X.shape
    dj_dw = np.zeros((n, ))
    dj_db = 0.0

    for i in range(m):
        err = (np.dot(X[i], w) + b) - y[i]
        for j in range(n):
            dj_dw[j] += err * X[i, j]
        dj_db += err
    
    dj_dw /= m
    dj_db /= m

    return dj_dw, dj_db

def compute_gradient_descent(X, y, w_in, b_in, cost, gradient, alpha, num_iters = 1000):
    J_history = []
    w = copy.deepcopy(w_in)
    b = b_in

    for i in range(num_iters):
        dj_dw, dj_db = gradient(X, y, w, b)
        w -= alpha * dj_dw
        b -= alpha * dj_db

        if i < 100000:
            J_history.append(cost(X, y, w, b))
        
        if i % math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost{J_history[-1]:8.2f}")
    
    return w, b, J_history

In [75]:
initial_w = np.zeros((X_train.shape[1]))
initial_b = 0
w_final, b_final, J_history = compute_gradient_descent(X_train, y_train, initial_w, initial_b, compute_cost, compute_gradient, alpha=6.5e-3, num_iters=10000)
print(f"Los pesos encontrados manualmente son: {w_final}")
print(f"El intercepto encontrado manualmente es: {b_final}")

Iteration    0: Cost   58.43
Iteration 1000: Cost    1.84
Iteration 2000: Cost    1.83
Iteration 3000: Cost    1.82
Iteration 4000: Cost    1.82
Iteration 5000: Cost    1.81
Iteration 6000: Cost    1.81
Iteration 7000: Cost    1.81
Iteration 8000: Cost    1.81
Iteration 9000: Cost    1.81
Los pesos encontrados manualmente son: [ 0.08351989 -0.18272994 -0.33906367  0.03666412  0.1477936   0.98121627]
El intercepto encontrado manualmente es: -1.5742111184990806


In [72]:
y_hat = np.zeros_like(y_test)

for i in range(X_test.shape[0]):
    y_hat[i] = np.dot(X_test[i], w_final) + b_final

In [79]:
for i in range(len(y_hat)):
    print(f"La nota real del estudiante número {i + 1} es: {y_test[i]}. El estimador calculado es: {y_hat[i]}")

La nota real del estudiante número 1 es: 14. El estimador calculado es: 11
La nota real del estudiante número 2 es: 13. El estimador calculado es: 13
La nota real del estudiante número 3 es: 8. El estimador calculado es: 5
La nota real del estudiante número 4 es: 11. El estimador calculado es: 10
La nota real del estudiante número 5 es: 10. El estimador calculado es: 10
La nota real del estudiante número 6 es: 0. El estimador calculado es: -1
La nota real del estudiante número 7 es: 10. El estimador calculado es: 8
La nota real del estudiante número 8 es: 15. El estimador calculado es: 15
La nota real del estudiante número 9 es: 11. El estimador calculado es: 10
La nota real del estudiante número 10 es: 16. El estimador calculado es: 16
La nota real del estudiante número 11 es: 0. El estimador calculado es: 0
La nota real del estudiante número 12 es: 12. El estimador calculado es: 11
La nota real del estudiante número 13 es: 7. El estimador calculado es: 6
La nota real del estudiante n

In [81]:
y_bar = y_test.mean()
s_res = 0.
s_tot = 0.
for i in range(len(y_test)):
    s_res += (y_hat[i] - y_test[i])**2
    s_tot += (y_test[i] - y_bar)**2

coef = 1 - (s_res/s_tot)
print(f"La precision lograda manualmente fue de: {coef}")

La precision lograda manualmente fue de: 0.7559487492373398


# sklearn Implementation

In [77]:
linear = linear_model.LinearRegression()
linear.fit(X_train, y_train)
acc = linear.score(X_test, y_test)
print(f"La precision de sklearn fue de: {acc}")
print(f"Los pesos encontrados con sklearn son: {linear.coef_}")
print(f"El intercepto encontrado con sklerarn es: {linear.intercept_}")

predict = linear.predict(X_test)
print(predict)

La precision de sklearn fue de: 0.7997674777423431
Los pesos encontrados con sklearn son: [ 0.10618557 -0.16570436 -0.3226161   0.03733081  0.1526025   0.98302086]
El intercepto encontrado con sklerarn es: -1.7282170164285056
[11.91440609 13.1115892   5.02298438 10.97656386 10.05733683 -1.71637245
  8.45666546 15.1739939  10.11283043 16.2952109  -0.21504405 11.6092011
  6.1603894   8.94498592 14.06457426 12.63085726 18.95213107 15.9728883
 12.91584905 13.51170829  9.45928279  7.20620831 12.94409374 12.302857
 11.16252069 12.76324655  9.82400656 10.68773999 12.15934061  9.08448655
  9.64315938  5.69169513 12.99051066 11.95527547  8.18922917 12.97536783
 12.93823241 14.52188937  9.42046382  6.29784906]
